In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()

drive=GoogleDrive(gauth)

In [0]:
train_clean = drive.CreateFile({'id': '1_y2mSjhcw0uaI_vTgrLdd0UScWIEsjIb'})
train_clean.GetContentFile('train_clean.csv')
test_clean = drive.CreateFile({'id': '1U0g_7u7g61UTywF0elODU06jyJ_wLfjF'})
test_clean.GetContentFile('test_clean.csv')

In [0]:
train = drive.CreateFile({'id': '1UAMpsYjI2161mmcs8MVsuEF-1okd43jT'})
train.GetContentFile('train.csv')
test = drive.CreateFile({'id': '17uPPXQbgzT772MCpsoddfEIYS95oGkD0'})
test.GetContentFile('test.csv')
sub = drive.CreateFile({'id': '1_48gjftaBNWfVd4DiMG4a7qQ-3Bv7P_N'})
sub.GetContentFile('sample_submission.csv')



In [4]:

############################imports##########################################


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

import tqdm
from sklearn.model_selection import train_test_split,GroupKFold, KFold

from sklearn.metrics import confusion_matrix, f1_score, plot_confusion_matrix,mean_squared_error
from keras.models import Model
import keras.layers as L
from keras.utils import to_categorical, plot_model

import json
from google.colab import files

from scipy import signal

import lightgbm as lgb

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def normalize(X):
  dataset = X.values
  data_mean = dataset.mean(axis=0)
  data_std = dataset.std(axis=0)
  dataset = (dataset-data_mean)/data_std
  return dataset


In [0]:
train_clean=pd.read_csv('train_clean.csv')
test_clean=pd.read_csv('test_clean.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


In [0]:
batch_separated=[];batch_separated_clean_train=[];batch_separated_clean_test=[]
#train_clean['signal']=normalize(train_clean.signal)
#test_clean['signal']=normalize(test_clean.signal)
single_batch_size=500000
num_of_batch_train=int(train.shape[0]/single_batch_size)
num_of_batch_test=int(test.shape[0]/single_batch_size)

#train_clean['group']=0;test_clean['group']=0
for i in range(num_of_batch_train):
 # train_clean['group'].iloc[i*single_batch_size:(i+1)*single_batch_size]=i
  temp_df=train.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated.append(temp_df)
  temp_df=train_clean.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated_clean_train.append(temp_df)
for i in range(num_of_batch_test):
  #test_clean['group'].iloc[i*single_batch_size:(i+1)*single_batch_size]=i
  temp_df=test_clean.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated_clean_test.append(temp_df)

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

#train=reduce_mem_usage(train)

# Feature Engineering

In [0]:
def feature_engineering(s,windows_roll,windows_ewm,windows_lag):
  ewm = pd.DataFrame();high_pass = pd.DataFrame();low_pass = pd.DataFrame();grads = pd.DataFrame();merged_features=pd.DataFrame()
  roll_stats = pd.DataFrame();lead_lag=pd.DataFrame();column_names=[]
  n_filts=10
  wns = np.logspace(-2, -0.3, n_filts)
  n_grads=4
  for window in windows_roll:
        roll_stats['roll_mean_' + str(window)] = s.rolling(window=window, min_periods=1).mean()
        roll_stats['roll_std_' + str(window)] = s.rolling(window=window, min_periods=1).std()
        roll_stats['roll_min_' + str(window)] = s.rolling(window=window, min_periods=1).min()
        roll_stats['roll_max_' + str(window)] = s.rolling(window=window, min_periods=1).max()
        roll_stats['roll_range_' + str(window)] = roll_stats['roll_max_' + str(window)] - roll_stats['roll_min_' + str(window)]
        roll_stats['roll_q10_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.10)
        roll_stats['roll_q25_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.25)
        roll_stats['roll_q50_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.50)
        roll_stats['roll_q75_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.75)
        roll_stats['roll_q90_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.90)
  roll_stats = roll_stats.fillna(value=0)
  #print(roll_stats.shape)
  roll_stats=roll_stats.reset_index(drop=True)
  for window in windows_lag:
    lead_lag['lag_'+str(window)]=s.shift(window)
    lead_lag['lead_'+str(window)]=s.shift(-1*window)
  lead_lag=lead_lag.fillna(0)
  lead_lag=lead_lag.reset_index(drop=True)


  g=s.values
  for i in range(n_grads):
    g = np.gradient(g)
    grads['grad_' + str(i+1)] = g
  grads=grads.fillna(0)
  grads=grads.reset_index(drop=True)
  
  for wn in wns:
    b, a = signal.butter(1, Wn=wn, btype='low')
    low_pass['lowpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, s.values)
    low_pass['lowpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, s.values)
  low_pass=low_pass.fillna(0)
  low_pass=low_pass.reset_index(drop=True)
  
  for wn in wns:
    b, a = signal.butter(1, Wn=wn, btype='high')
    high_pass['hihgpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, s.values)
    high_pass['hihgpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, s.values)  

  high_pass=high_pass.fillna(0)
  high_pass=high_pass.reset_index(drop=True)
  for window in windows_ewm:
    ewm['ewm_mean_' + str(window)] = s.ewm(span=window, min_periods=1).mean()
    ewm['ewm_std_' + str(window)] = s.ewm(span=window, min_periods=1).std()
  ewm=ewm.fillna(0)
  ewm=ewm.reset_index(drop=True)
  merged_features=pd.concat([roll_stats,lead_lag,grads,ewm,low_pass,high_pass],axis=1,ignore_index=True) 
  #merged_features=merged_features.append([roll_stats,lead_lag,grads,ewm,low_pass,high_pass],)
  #print(grads.shape,low_pass.shape,high_pass.shape,ewm.shape,lead_lag.shape,roll_stats.shape)
  #print("aaa")

  column_names=list(roll_stats.columns)+list(lead_lag.columns)+list(grads.columns)+list(ewm.columns)+list(low_pass.columns)+list(high_pass.columns)
  idxx=[list(roll_stats.columns),list(lead_lag.columns),list(grads.columns),list(ewm.columns),list(low_pass.columns),list(high_pass.columns)]
    #column_names=list(lead_lag.columns)+list(grads.columns)+list(ewm.columns)+list(low_pass.columns)+list(high_pass.columns)
  #print(merged_features.shape)
  return merged_features,column_names,idxx


In [10]:
windows_roll=[10,50,100,500];windows_ewm=[10,50,100];windows_lag=[1,2,3];new_features=pd.DataFrame()
temp_df=pd.DataFrame()

for i in tqdm.tqdm(range(num_of_batch_train)):  
  temp_df,columns,idxx=feature_engineering(batch_separated_clean_train[i].signal,windows_roll,windows_ewm,windows_lag)
  #new_features=pd.concat([new_features,temp_df],axis=0,ignore_index=True)
  new_features=new_features.append(temp_df)
  
col_change=dict(zip(temp_df.columns,columns))
new_features=reduce_mem_usage(new_features)
new_features=new_features.reset_index(drop=True)
train=pd.concat([train_clean,new_features],axis=1)
train.rename(columns=col_change,inplace=True)

train['sig2']=train['signal']**2


100%|██████████| 10/10 [01:18<00:00,  7.81s/it]


Mem. usage decreased to 953.67 Mb (74.2% reduction)


In [11]:
new_features=pd.DataFrame();temp_df=pd.DataFrame()

for i in tqdm.tqdm(range(num_of_batch_test)):  
  temp_df,columns,idxx=feature_engineering(batch_separated_clean_test[i].signal,windows_roll,windows_ewm,windows_lag)
  #new_features=pd.concat([new_features,temp_df],axis=0,ignore_index=True)
  new_features=new_features.append(temp_df)
  
col_change=dict(zip(temp_df.columns,columns))
new_features=reduce_mem_usage(new_features)
new_features=new_features.reset_index(drop=True)
test=pd.concat([test_clean,new_features],axis=1)
test.rename(columns=col_change,inplace=True)
test['sig2']=test['signal']**2

100%|██████████| 4/4 [00:28<00:00,  7.24s/it]


Mem. usage decreased to 381.47 Mb (74.2% reduction)


# Modeling

In [12]:

y=train.open_channels
X=train.drop(['time','open_channels'],axis=1)

#X=X[new_cols]
FEATURES=X.columns
x_train, x_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2)
X.shape

(5000000, 98)

In [0]:
test=test.drop('time',axis=1)

In [14]:
def create_mpl(shape):
    '''
    Returns a keras model
    '''
    print("bb")
    
    X_input = L.Input(shape)
    print("aa")
    X = L.Dense(150, activation='relu')(X_input)
    #X = L.Dropout(0.25)(X)
    #X = L.Dense(150, activation='relu')(X)

    X = L.Dense(125, activation='relu')(X)
    #X = L.Dropout(0.25)(X)
    X = L.Dense(100, activation='relu')(X)
    X = L.Dense(75, activation='relu')(X)

    X = L.Dense(50, activation='relu')(X)
    X = L.Dense(25, activation='relu')(X)
    X = L.Dense(11, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    print(shape)
    return model


mlp = create_mpl(x_train[0].shape)
mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
print(mlp.summary())

def get_class_weight(classes):
    '''
    Weight of the class is inversely proportional to the population of the class
    '''
    hist, _ = np.histogram(classes, bins=np.arange(12)-0.5)
    class_weight = hist.sum()/hist
    
    return class_weight

class_weight = get_class_weight(y_train)

bb
aa
(98,)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 98)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               14850     
_________________________________________________________________
dense_2 (Dense)              (None, 125)               18875     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               12600     
_________________________________________________________________
dense_4 (Dense)              (None, 75)                7575      
_________________________________________________________________
dense_5 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_6 (Dense)              (None, 25)        

In [0]:
import tensorflow as tf

class MacroF1(tf.keras.callbacks.Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = targets
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis = -1)
        score = f1_score(self.targets, pred, average = 'macro')
        print(f'F1 Macro Score: {score:.5f}')



In [0]:
folds=5;epochs=40
kf = KFold(n_splits=folds, shuffle=True, random_state=667)
fold=1
preds_ = np.zeros((len(test), 11))

for train_index, val_index in kf.split(X, y):
      print(f'====== Fold {fold:0.0f} of {folds} ======')

      train_X = X.iloc[train_index]
      val_X = X.iloc[val_index]
      train_y = y.iloc[train_index]
      val_y = y.iloc[val_index]
    
      mlp.fit(x=train_X, y=train_y, epochs=epochs, batch_size=1024, class_weight=class_weight,callbacks=[MacroF1(mlp,x_test,y_test)])

      f1=f1_score(val_y,np.argmax(mlp.predict(val_X),axis=-1),average='macro')
      print(f"F1-Score for classifier of fold {fold} "," : ",f1)
      preds_proba=mlp.predict(test) / folds
      preds_ += preds_proba
    
      fold+=1



In [0]:
sub=pd.read_csv('sample_submission.csv')
sub['open_channels'] = np.argmax(preds_, axis = 1).astype(int)
sub.to_csv('submission_mlp.csv', index=False, float_format='%.4f')
files.download('submission_mlp.csv')